In [ ]:
import tensorflow as tf

data_dir = "/kaggle/input/traffic-sign-classification-datasets/traffic_Data/DATA"


In [ ]:
import os

dataset_path = '/kaggle/input/traffic-sign-classification-datasets/traffic_Data/DATA'
dataset_size = sum(os.path.getsize(os.path.join(dataset_path, f)) for f in os.listdir(dataset_path))

print(f"Dataset size: {dataset_size} bytes")


In [ ]:
train_dir="/kaggle/input/traffic-sign-classification-datasets/traffic_Data/train"

In [ ]:
test_dir="/kaggle/input/traffic-sign-classification-datasets/traffic_Data/test"

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1.0/255)#initialize train generator 
                                 
test_datagen = ImageDataGenerator(rescale = 1.0/255.) #initialize validation generator

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(128,128),batch_size=32,class_mode='categorical')

#validation_generator = valid_datagen.flow_from_directory(validation_ds, target_size=(128,128),batch_size=32,class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(128,128),batch_size=32,class_mode='categorical')

In [ ]:
import os

train_dir = '/kaggle/input/traffic-sign-classification-datasets/traffic_Data/train'

# Count the number of images in each subdirectory
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)
    if os.path.isdir(class_dir):
        num_images = len(os.listdir(class_dir))
        print("Class {}: {} images".format(class_name, num_images))


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from skimage.transform import resize

# Define the path to your train folder containing the images
train_dir = '/kaggle/input/traffic-sign-classification-datasets/traffic_Data/train'

# Define the number of images you want to generate for each class
num_samples = 100

# Define the parameters for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest'
)

# Get a list of the classes in the train folder
classes = sorted(os.listdir(train_dir))

# Generate balanced samples for each class
for class_name in classes:
    class_dir = os.path.join(train_dir, class_name)
    num_existing_samples = len(os.listdir(class_dir))
    if num_existing_samples < num_samples:
        num_to_generate = num_samples - num_existing_samples
        print(f"Generating {num_to_generate} samples for class {class_name}")
        image_files = [os.path.join(class_dir, f) for f in os.listdir(class_dir)]
        image_array = np.array([resize(plt.imread(f), (128, 128)) for f in image_files])
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)
        flow = datagen.flow(image_array, batch_size=32, save_to_dir=class_dir, save_prefix=class_name, save_format='png')
        for i in range(num_to_generate // 32):
            flow.next()
        remaining = num_to_generate % 32
        if remaining > 0:
            flow = datagen.flow(image_array, batch_size=remaining, save_to_dir=class_dir, save_prefix=class_name, save_format='png')
            flow.next()
    else:
        print(f"Class {class_name} is already balanced")


In [ ]:
img=train_generator[0]
img

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Softmax
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def vgg19_model(input_shape=(128, 128, 3), num_classes=58):
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 5
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Classification block
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    return model


In [ ]:
# create the model
model = vgg19_model()

# print the model summary
model.summary()

In [ ]:
type(data_dir)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])  

In [ ]:
har = model.fit(train_generator, validation_data=test_generator, epochs=5,batch_size=32)

In [ ]:
import numpy as np 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
file_list = os.listdir(test_dir)
num_files = len(file_list)
batch_size = 32
num_batches = int(num_files / batch_size)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(har.history)
df.head()

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(har.history['accuracy'])
plt.plot(har.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(har.history['loss'])
plt.plot(har.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001)
acc = tf.keras.metrics.SparseCategoricalAccuracy()
los = tf.keras.losses.SparseCategoricalCrossentropy()


model.compile(
    
    optimizer = opt,
    loss = los,
    metrics=acc
)

In [ ]:

loss_plot = df.plot(y='loss', title='Loss vs Epochs', legend=False)
loss_plot.set(xlabel='Epochs', ylabel='Loss')

In [ ]:
acc_plot = df.plot(y='accuracy', title='Accuracy vs Epochs', legend=False)
acc_plot.set(xlabel='Epochs', ylabel='Accuracy')

In [ ]:

import cv2

t=cv2.imread('/kaggle/input/traffic-sign-classification-datasets/traffic_Data/test/0/000_0011.png')
plt.imshow(t)

In [ ]:

import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
testimg=cv2.resize(t,(128,128))
testimg=img_to_array(testimg)/255
h=np.expand_dims(testimg,axis=0)
r=model.predict(h)
classnames=["0","1","2","3","4","5","6","7","8","9","10",
            "11","12","13","14","15","16","17","18","19","20",
            "21","22","23","24","25","26","27","28","29","30",
            "31","32","33","34","35","36","37","38","39","40",
            "41","42","43","44","45","46","47","48","49","50"
            "51","52","53","54","55","56","57"]
ypred=classnames[np.argmax(r)]
ypred

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test=ImageDataGenerator(rescale=1./255)

In [ ]:

import pathlib

In [ ]:
labels = ["0","1","2","3","4","5","6","7","8","9","10",
            "11","12","13","14","15","16","17","18","19","20",
            "21","22","23","24","25","26","27","28","29","30",
            "31","32","33","34","35","36","37","38","39","40",
            "41","42","43","44","45","46","47","48","49","50"
            "51","52","53","54","55","56","57"]
img_size = 224
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)